In [26]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter1d
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

In [68]:
def cargar_datos(directorio):
    data = []
    etiquetas = []
    for carpeta in os.listdir(directorio):
        subdirectorio = os.path.join(directorio, carpeta, 'csv')
        print(carpeta)
        for archivo in glob.glob(os.path.join(subdirectorio, '*.csv')):
            df = pd.read_csv(archivo)
            data.append(df)
            etiquetas.append(carpeta)
    return data, etiquetas

In [30]:
def normalizar_coordenadas(df):
    # Obtener las coordenadas promedio de las caderas
    cadera_derecha = df[df['landmark_index'] == 23][['x', 'y', 'z']].mean()
    cadera_izquierda = df[df['landmark_index'] == 24][['x', 'y', 'z']].mean()
    cadera_centro = (cadera_derecha + cadera_izquierda) / 2
    
    # Normalizar restando las coordenadas de la cadera central
    df['x'] -= cadera_centro['x']
    df['y'] -= cadera_centro['y']
    df['z'] -= cadera_centro['z']
    
    # Normalización por el tamaño del torso (distancia entre hombros 11 y 12)
    hombro_derecho = df[df['landmark_index'] == 11][['x', 'y', 'z']].mean()
    hombro_izquierdo = df[df['landmark_index'] == 12][['x', 'y', 'z']].mean()
    torso_tamano = np.linalg.norm(hombro_derecho - hombro_izquierdo)
    
    df['x'] /= torso_tamano
    df['y'] /= torso_tamano
    df['z'] /= torso_tamano
    
    return df

In [32]:
# Filtrar las coordenadas
def filtrar_datos(df):
    df['x'] = gaussian_filter1d(df['x'], sigma=2)
    df['y'] = gaussian_filter1d(df['y'], sigma=2)
    df['z'] = gaussian_filter1d(df['z'], sigma=2)
    return df

In [34]:
# Calcular el ángulo entre tres puntos
def calcular_angulo(p1, p2, p3):
    vector1 = p1 - p2
    vector2 = p3 - p2
    cos_theta = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    angulo = np.arccos(np.clip(cos_theta, -1.0, 1.0))
    return np.degrees(angulo)

In [56]:
# Generación de características agregadas
def generar_caracteristicas(df):
    # Articulaciones clave: hombros, codos, y caderas
    articulaciones_clave = [11, 12, 13, 14, 15, 16, 23, 24]
    
    velocidades = []
    angulos_codo_derecho = []
    angulos_codo_izquierdo = []
    angulos_tronco = []
    
    for i in range(1, len(df)):
        fila_actual = df[df['frame'] == i]
        fila_anterior = df[df['frame'] == i - 1]
        
        # Calcular velocidades
        for articulacion in articulaciones_clave:
            actual = fila_actual[fila_actual['landmark_index'] == articulacion]
            anterior = fila_anterior[fila_anterior['landmark_index'] == articulacion]
            
            if not actual.empty and not anterior.empty:
                vel_x = actual['x'].values[0] - anterior['x'].values[0]
                vel_y = actual['y'].values[0] - anterior['y'].values[0]
                vel_z = actual['z'].values[0] - anterior['z'].values[0]
                velocidad = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
                velocidades.append(velocidad)

        # Calcular ángulos
        hombro_derecho = fila_actual[fila_actual['landmark_index'] == 11][['x', 'y', 'z']].values
        codo_derecho = fila_actual[fila_actual['landmark_index'] == 13][['x', 'y', 'z']].values
        muneca_derecha = fila_actual[fila_actual['landmark_index'] == 15][['x', 'y', 'z']].values
        
        if hombro_derecho.size > 0 and codo_derecho.size > 0 and muneca_derecha.size > 0:
            angulo_codo_derecho = calcular_angulo(hombro_derecho[0], codo_derecho[0], muneca_derecha[0])
            angulos_codo_derecho.append(angulo_codo_derecho)
        
        hombro_izquierdo = fila_actual[fila_actual['landmark_index'] == 12][['x', 'y', 'z']].values
        codo_izquierdo = fila_actual[fila_actual['landmark_index'] == 14][['x', 'y', 'z']].values
        muneca_izquierda = fila_actual[fila_actual['landmark_index'] == 16][['x', 'y', 'z']].values
        
        if hombro_izquierdo.size > 0 and codo_izquierdo.size > 0 and muneca_izquierda.size > 0:
            angulo_codo_izquierdo = calcular_angulo(hombro_izquierdo[0], codo_izquierdo[0], muneca_izquierda[0])
            angulos_codo_izquierdo.append(angulo_codo_izquierdo)
        
        cadera_centro = ((fila_actual[fila_actual['landmark_index'] == 23][['x', 'y', 'z']].values +
                          fila_actual[fila_actual['landmark_index'] == 24][['x', 'y', 'z']].values) / 2)
        
        if hombro_derecho.size > 0 and cadera_centro.size > 0 and hombro_izquierdo.size > 0:
            angulo_tronco = calcular_angulo(hombro_derecho[0], cadera_centro[0], hombro_izquierdo[0])
            angulos_tronco.append(angulo_tronco)
    
    # Calcular estadísticas de cada característica
    caracteristicas = [
        np.mean(velocidades), np.std(velocidades),
        np.mean(angulos_codo_derecho), np.std(angulos_codo_derecho),
        np.mean(angulos_codo_izquierdo), np.std(angulos_codo_izquierdo),
        np.mean(angulos_tronco), np.std(angulos_tronco)
    ]
    
    return caracteristicas

In [38]:
def preprocesar_y_extraer(data):
    datos_procesados = []
    for df in data:
        df = normalizar_coordenadas(df)
        df = filtrar_datos(df)
        caracteristicas = generar_caracteristicas(df)
        datos_procesados.append(caracteristicas)
    return datos_procesados

In [40]:
# Entrenar el modelo
def entrenar_modelo(datos_procesados, etiquetas):
    X_train, X_test, y_train, y_test = train_test_split(datos_procesados, etiquetas, test_size=0.2, random_state=42)
    modelo = RandomForestClassifier(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    print(classification_report(y_test, y_pred))
    return modelo

In [58]:
# Guardar el modelo entrenado
def guardar_modelo(modelo, nombre_archivo):
    joblib.dump(modelo, nombre_archivo)

In [71]:
directorio_datos = './datos_Mateo'
data, etiquetas = cargar_datos(directorio_datos)
datos_procesados = preprocesar_y_extraer(data)

caminar hacia atras
caminar hacia delante
pararse
Sentarse


In [73]:
modelo = entrenar_modelo(datos_procesados, etiquetas)

                       precision    recall  f1-score   support

  caminar hacia atras       1.00      1.00      1.00         2
caminar hacia delante       1.00      1.00      1.00         1
              pararse       1.00      1.00      1.00         1

             accuracy                           1.00         4
            macro avg       1.00      1.00      1.00         4
         weighted avg       1.00      1.00      1.00         4

